# Data Cleaning

### Objectives

1. Load in the raw dataset for further analysis and preporation for data cleaning.
2. Revisit correlation and perform a PPS (Predictive Power Score) Study.
3. Inspect the data set for missing values, with a focus on the features highlighted in the DataAnalysis Notebook.
4. Having dealt with any missing values split the now clean data set into Train and Test sets.
5. Push these new data sets to the repo ready for feature engineering.

### Inputs

1. Our house_price_records data that we collected in our DataCollection Notebook, found at inputs/datasets/raw/house-price-20211124T154130z/house-price/house_price_records.csv 

### Additional Comments

* This notebook was designed and follows the principles set out by Code Institute in the Predictive Analytics lessons and Walkthrough projects. The code written in this work book has taken influence from these lessons and projects but has been modiefied, in some cases such as the graphical design, heavily modified by myself in order to suit the needs for this project.

___

## Change working Directory

* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir


We want to make the parent of the current directory the new current directory

* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

## Load Collected Data and Import Necessary Packages

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer, MeanMedianImputer, DropMissingData
from sklearn.pipeline import Pipeline

* Firstly load both datasets ready for data evalustaion and cleaning. 

In [ ]:
df = (pd.read_csv("inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv"))
df.head()

In [ ]:
df_inherited = (pd.read_csv("inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/inherited_houses.csv"))
df_inherited.head()

___

## Data Exploration

Having loaded our data we want to assess the distribution of the variables that contain missing data.

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

We can then run the Pandas Profile Report again, however this time we have singled out the columns that contain missing features in order to get a better understanding of their distribution.

In [ ]:
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

___

### PPS Analysis and Revisit of Correlation

* The Code utilised here was taken from the Code Isntitute lessons focused around PPS (Predictive Power score).
* We are utilising heatmaps here to review the PPS and correlation between our target Sales Price and other features.

In [ ]:
%matplotlib inline

def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)


* Calculate the PPS (Predictive Power Score)

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

* Below is the calling of our above function, we have set the PPS_threshhold at 0.2 in order to get a visualisation of features that have a relativly weak to strong predictive power. A score of less then 0.2 would mean a very weak predictive power and would arguably not be worth our time investigating.
* As seen in the previous notebook the features we highlighted as potentail features of importance showed a strong corralation, as such the correlation threshhold has been set to 0.5 and we would hope to see all 6 of these features present in the heat maps.

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.5, PPS_Threshold =0.2,
                  figsize=(12,10), font_annot=10)

___

## Further Data Analysis
### Missing data analysis

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data


In [ ]:
EvaluateMissingData(df)

In [ ]:
EvaluateMissingData(df_inherited)

##### Results of missing data evaluation

* As we can see we have 9 columns that contain missing data, with both EnclosedPorch and WoodDeckSF having a hugely significant portion of their data missing.
* The other columns do not have nearly as much missing data as the previously mentioned two, however the amount if missing data is still an issue.
* The brightside here is that none of the features that were intially flagged as potentially important in the previous notebook appear in this list.

#### Effects of data cleaning methods

* In the previous notebook we completed some very simple data cleaning just to get a rough look at what effect it would have on the features that we identefied as potentally important. We wanted to see there that if after some simple cleaning and engineering whether those features remained the most important.
* The above PPS and correlation tests have then reinforced that based on the raw data we collected the top features that look to be able to predict SalePrice are:
    1. A large 1st floor square footage. (1stFlrSF)
    2. A large garage square footage. (GarageArea)
    3. A large above grade (ground) living area square footage. (GrLivArea)
    4. A high overall quality. (OverallQual)
    5. A large basement square footage. (TotalBsmtSF)
    6. A house built more recently. (YearBuilt)


* What we shall now look at is a more in-depth look into the effects of different data cleaning methods, The code below has been taken from the Code Institute Lessons relating to Predictive Analytics.
    * The below function assess the effect of different data cleaning methods such replacing missing numerical values with the Mean or the Median, and replacing Categorical values with 'Missing' or the most frequently seen value. 
    * Once we have gone through and cleaned our data producing a new cleaned dataframe we shall use this function to evaluate any significant changes to the original. 

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt

def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate the plot number
  
  # Distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over the variables, 
    # first on variables that you applied to the method
    # if the variable is numerical plot a histogram, if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

___

## Data Cleaning

* We can start the cleaning process by returning to an earlier function and pulling our columns that contain missing data.

In [ ]:
EvaluateMissingData(df)

* As previously stated both EnclosedPorch and WoodDeckSF have an extreamly high percentage of null values. It is therefore not viable to attempt to impute any values in place of these, for this reason we shall be dropping these columns entirely from our dataset.
* LotFrontage and GarageFinish also have a high percentage of null values. Both above 10%. They shall not be immediatly dropped without evaluation however as they may still provide insights after data cleaning.
* The remaining 5 variables will all be cleaned and evaluated using the function defined above.

* It is my assumption that due to the fact none of these variables have been flagged in any data analysis as potentially useful for predicting SalesPrice that after data cleaning they will still prove to be unimportant, however we can not confirm this assumption without performing the relevent data cleaning and subsequent evaluation. The only 2 exceptoions to this are EnclosedPorch and WoodDeckSF. Both of which are float64 variables with 90% null values meaning any mean, median or arbitary number imputation would not be close to a relistic representation of the data and will not aid in our predictions and could even have a negative impact if any.

#### Data cleaning we will try initially
1. Drop - ['EnclosedPorch', 'WoodDeckSF' ]
2. Arbitary Number Imputer - ['2ndFlrSF', 'BedroomAbvGr', 'MasVnrArea'] 
3. Impute the Mean - ['LotFrontage', 'GarageYrBlt' ]
4. Impute 'Missing' - ['GarageFinish', 'BsmtFinType1' ]

___

## Splt the cleaned data

The first step when cleaning is to create a copy of our dataset.

In [ ]:
df_clean = df.copy()
print(df_clean.shape)

We then split our original dataset into train and test sets.

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

We can now evaluate the spread of missing values with in the test set that we have just created.

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

* Upon reviweing the test set we can see the split is extrealy similar to the split of the raw data. As expected the same 9 variables have been flagged with a slight decrease in the the percantage of missing data, however we still see theat EnclosedPorch and WoodDeckSF have 90.41% and 88.53% missing data respectivly so we shall continue to work on the assumption that they will be of no help in answering the business requiremets and will be dropped due to this.

___

### Step 1 of data cleaning

* As stated above we want to drop EnclosedPorch and WoodDeckSF as they have a large majority of null values and will likely not effect our predictions in any meaningful way.
* In order to achieve this we will run the following code, utilising the DropFeatures meathod from feature_engine and will apply this change to both the train and test set.

In [ ]:
from feature_engine.selection import DropFeatures
variables = ['EnclosedPorch', 'WoodDeckSF']
imputer = DropFeatures(features_to_drop=variables)
imputer.fit(TrainSet)
TrainSet, TestSet = imputer.transform(TrainSet), imputer.transform(TestSet)
TestSet.head()

In [ ]:
df_clean = imputer.transform(df_clean)

We can check again that our code worked by placing all columns with null values into a list, as we can see EnclosedPorch and WoodDeckSF are no longer present as they have been dropped.

In [ ]:

null_variables = TrainSet.columns[TrainSet.isnull().any()].tolist()
print(null_variables)

In [ ]:
EvaluateMissingData(df_clean)

___

### Step 2 of Data cleaning

As stated above we have a large list of columns all of which were float64 variables, that we, as a base defined imputing the mean as a potential solution. However upon inspection of the data, we can deduce that imputing the mean my not be the most optimum solution.
The float64 variables are: ['2ndFlrSF', 'BedroomAbvGr', 'GarageYrBlt', 'LotFrontage', 'MasVnrArea'] 

After looking closly at the raw data set we can make some assumptions about the missing data which will help in our data cleaning.
* The first thing I noticed when looking more deeply into our data is that 2ndFlrSF has a very large amount of 0 values (626 rows). We can assume this is due to the propertty being a bungalow and therefore having no second floor area. Further more the null value count is relativy low (60 rows), for this reason we can safely assume that if we convert our null values to 0 values we will not negatively impact our data.
* Secondly BedroomAbvGr is a grading system of bedrooms from 0-8. On first impressions it looks like there is no accurate way to remove the null values, our heat map shows that there are no other variables with the ability to predict BedroomAbvGr and imputing the mean or median would be inaccurate and could negatively effect our predictions. However when we look at our Inherited_df (the data we want to predict). We can see that all the houses have an BedroomAbvGr > 0. Due to this we can set all our null values to 0 using ArbitaryNumberImputer just like 2nfFlrSF.
* Thirdly MasVnrArea follows a very similar trend to 2ndFlrSF. It has a very high count of 0 values. Which to me indicates that these houses have no Masonry Veneer. Furthermore in terms of the null value count there are only 8 rows with null values. Due to these reasons and the fact that the heat map showed no predictive power for MasVnrArea I am confident we may also impute 0 for null values in this field without negatively impacting our predictions.

In [ ]:
TrainSet['2ndFlrSF'].value_counts().sort_index()

In [ ]:
TrainSet['BedroomAbvGr'].value_counts().sort_index()

In [ ]:
TrainSet['MasVnrArea'].value_counts().sort_index()

In [ ]:
pipeline = Pipeline([
    ('2ndFlrSF_BedrromAbvGr_MasVnrArea', ArbitraryNumberImputer(arbitrary_number=0, variables=['2ndFlrSF', 'BedroomAbvGr', 'MasVnrArea']))
])
pipeline.fit(TrainSet)
TrainSet, TestSet = pipeline.transform(TrainSet), pipeline.transform(TestSet)
df_clean = pipeline.transform(df_clean)
EvaluateMissingData(TrainSet)

As we can see when we evaluate missing data in our train set 2ndFlrSF, BedroomAbvGr and MasVnrArea are no longer present. To double check our pipeline worked we will also check the missing values in our clean dataframe.

In [ ]:
EvaluateMissingData(df_clean)

___

### Step 3 of Data cleaning

As we can see we have 4 columns left that contain null values and require cleaning. The next one that I believe will be easily sorted is LotFrontage.

In [ ]:
TrainSet['LotFrontage'].value_counts().sort_index()

Looking at the LotFrontage data we can see it contains 212 rows of missing data. (Just under 18%)
The values range from 21.0 to 313.0. If we go back and look at the Pandas Profiling report we can see that the mean is 70.05 if we round to 2 decimal places and it has no 0 values. My first impression is to use to Impute the mean using MeanMedainImputer to all null values. Again we can see from the heat map LotFrontage holds no predictive power so we shouldn't have to worry about negative impacts to our predictions. 

In [ ]:
mean_imputer = MeanMedianImputer(imputation_method='mean', variables=['LotFrontage'])
mean_imputer.fit(TrainSet)
TrainSet, TestSet = mean_imputer.transform(TrainSet), mean_imputer.transform(TestSet)
df_clean = mean_imputer.transform(df_clean)
EvaluateMissingData(TrainSet)


Again upon inspection we can see our last 3 variables that contain null values, indicting our code worked on LotFrontage. We will test again though on the clean dataframe.

In [ ]:
EvaluateMissingData(df_clean)

___

### Step 4 of Data cleaning

The next variable to look at is BsmtFinType1 which is an object variable refering to the state of the basement that is finished. The first thing to do is inspect the varible itself. We already know that within our test set there are 89 rows containing missing data. We also know that from reviewing the heatmap that BsmtFinSF1 could be used to predict BsmtFinType1. We also have a lot of other variables that refer to the basement. For example TotalBsmtSF, it stands to reason that any house with a TotalBsmtSF that equals 0 does not have a basement. Therefore The BsmtFinType1 should equal 'None', we can query this and make sure this is true.

In [ ]:

TrainSet['BsmtFinType1'].value_counts().sort_index()

In [ ]:
TrainSet[TrainSet['BsmtFinType1'].isna()].query('TotalBsmtSF==0').sort_values(by=['TotalBsmtSF'])

As we can see 3 properties have 0 TotalBsmtSF and missing data, we can impute 'None' for these values.

In [ ]:
train_query_condition = (TrainSet.TotalBsmtSF == 0) & (TrainSet['BsmtFinType1'].isnull())
TrainSet['BsmtFinType1'] = np.where(train_query_condition, 'None', TrainSet['BsmtFinType1'])

test_query_condition = (TestSet.TotalBsmtSF == 0) & (TestSet['BsmtFinType1'].isnull())
TestSet['BsmtFinType1'] = np.where(test_query_condition, 'None', TestSet['BsmtFinType1'])

df_clean_query_condition = (df_clean.TotalBsmtSF == 0) & (df_clean['BsmtFinType1'].isnull())
df_clean['BsmtFinType1'] = np.where(df_clean_query_condition, 'None', df_clean['BsmtFinType1'])

In [ ]:
TrainSet[TrainSet['BsmtFinType1'].isna()].query('TotalBsmtSF==0').sort_values(by=['TotalBsmtSF'])

In [ ]:
TrainSet['BsmtFinType1'].isna().sum()

As we can see from our two tests, we have imputed None into our 3 rows and they no longer appear when we query TotalBsmtSF = 0 and null values. we can also see our total rows with null values has decreased by 3 to a total of 86.

In a similar method to our previous 3 rows, we shall query the null values of BsmtFinType1 against BsmtFinSF1. The aformentioned variable describes the surface area of the basement that is finished. if the value of this is 0 then we can conclude that the basment is unfinished and therefore we can impute 'Unf' into our null values.

In [ ]:
TrainSet[TrainSet['BsmtFinType1'].isna()].query('BsmtFinSF1==0').sort_values(by=['BsmtFinSF1'])

In [ ]:
unf_df = TrainSet[TrainSet['BsmtFinType1'].isna()].query('BsmtFinSF1==0').sort_values(by=['BsmtFinSF1'])
print(unf_df.shape)

As we can see we have 27 rows where we can conclude that the Basment is unfinished.

In [ ]:
train_query_condition = (TrainSet.BsmtFinSF1 == 0) & (TrainSet['BsmtFinType1'].isnull())
TrainSet['BsmtFinType1'] = np.where(train_query_condition, 'Unf', TrainSet['BsmtFinType1'])

test_query_condition = (TestSet.BsmtFinSF1 == 0) & (TestSet['BsmtFinType1'].isnull())
TestSet['BsmtFinType1'] = np.where(test_query_condition, 'Unf', TestSet['BsmtFinType1'])

df_clean_query_condition = (df_clean.BsmtFinSF1 == 0) & (df_clean['BsmtFinType1'].isnull())
df_clean['BsmtFinType1'] = np.where(df_clean_query_condition, 'Unf', df_clean['BsmtFinType1'])

In [ ]:
TrainSet[TrainSet['BsmtFinType1'].isna()].query('BsmtFinSF1==0').sort_values(by=['TotalBsmtSF'])

In [ ]:
TrainSet['BsmtFinType1'].isna().sum()

Again we can see that our null values are dropping as we are imputing values. and we are left with 59 rows with null values.
* The question is now how do we define these null values. As stated earlier there are other factors that can be used to predict BsmtFinType1. However the work needed to produce a model that could accurately predict BsmtFinType1 would be great and would it add value to our business requirements? Looking at the heatmap and correlation we can see that BsmtFinType1 has no correlation to SalesPrice and does not hold any predictive power towards it. Therefore I assume that there is no real need to get a accurate value for these remaining rows. For this reason we will simply use the CatergoricalImputer to place 'Unknown' in place of these values.


In [ ]:
unknown_imputer = CategoricalImputer(
    imputation_method='missing', fill_value='Unknown', variables='BsmtFinType1'
    )
unknown_imputer.fit(TrainSet)
TrainSet, TestSet, df_clean = unknown_imputer.transform(TrainSet), unknown_imputer.transform(TestSet), unknown_imputer.transform(df_clean)

In [ ]:
EvaluateMissingData(TrainSet)

As we can see we have our final 2 variables left to clean. These have been left till last as they both refer to the garage and like with the BsmtFinType1 we may see links that help us impute accurate data for the null values.

___

### Step 5 of Data cleaning

The first variable out of the final two we shall look at is GarageFinish, which is an object variable that refers to the state of the garage. First we shall inspect the data. We already know from above that there are 131 rows with missing data.

In [ ]:
TrainSet['GarageFinish'].value_counts().sort_index()

Similarly to the method used when cleaning BsmtfinType1, we shall use another variable to infer information about the GarageFinish. If we look at GarageArea, we can assume that if GarageArea equals 0 then GarageFinish should equal 'None'. 

In [ ]:
TrainSet[TrainSet['GarageFinish'].isna()].query('GarageArea==0').sort_values(by=['GarageArea'])

We can see we have 5 rows where GarageFinish is null and GarageArea equals 0. Like before we shall impute 'None' into these rows.

In [ ]:
train_query_condition = (TrainSet.GarageArea == 0) & (TrainSet['GarageFinish'].isnull())
TrainSet['GarageFinish'] = np.where(train_query_condition, 'None', TrainSet['GarageFinish'])

test_query_condition = (TestSet.GarageArea == 0) & (TestSet['GarageFinish'].isnull())
TestSet['GarageFinish'] = np.where(test_query_condition, 'None', TestSet['GarageFinish'])

df_clean_query_condition = (df_clean.GarageArea == 0) & (df_clean['GarageFinish'].isnull())
df_clean['GarageFinish'] = np.where(df_clean_query_condition, 'None', df_clean['GarageFinish'])

In [ ]:
TrainSet[TrainSet['GarageFinish'].isna()].query('GarageArea==0').sort_values(by=['GarageArea'])

In [ ]:
TrainSet['GarageFinish'].isna().sum()

As we can see our count of null values has reduced by 5 indicating our code has worked. We can double check that 'None' has increased by 5 by running the following code.

In [ ]:
TrainSet['GarageFinish'].value_counts().sort_index()

Unfortuantly there are no other variables that we can use that will give insight into the GarageFinish. And after a review of the PPS heatmap we can see that GarageFinihs holds no predictive power on the business requirement (SalesPrice). For this reason we shall again impute 'Unknown' to the rest of the null values for GarageFinish. 

In [ ]:
unknown_imputer = CategoricalImputer(
    imputation_method='missing', fill_value='Unknown', variables='GarageFinish'
    )
unknown_imputer.fit(TrainSet)
TrainSet, TestSet, df_clean = unknown_imputer.transform(TrainSet), unknown_imputer.transform(TestSet), unknown_imputer.transform(df_clean)

In [ ]:
TrainSet['GarageFinish'].value_counts().sort_index()

In [ ]:
EvaluateMissingData(TrainSet)

As we can see we now have our 'Unknown' values in GarageFinish, and upon evaluating the missing data the only variable left is GarageYrBlt.

___

### Step 6 of Data cleaning

The final Variable to be cleaned is GarageYrBlt, which refers to the year in which the garage is built, it is a float64 variable with 58 null values.

In [ ]:
TrainSet['GarageYrBlt'].value_counts().sort_index()

We can make the assumption that a null value is GarageyrBlt indicates that there is no garage. If that is the case then GarageFinish should equal None at the same instance that we have our null values. We can test this by running the code below.

In [ ]:
query_df = TrainSet[TrainSet['GarageYrBlt'].isna()].query('GarageFinish=="None"').sort_values(by=['GarageFinish'])
print(query_df.shape)

As we can see the code has returned the shape of the dataframe produced by our query. The shape shows that there are 58 rows where GarageFinish is 'None' and GarageYrBlt is null proving our assumption.
We shall use Arbitary Number Imputer to place 0 into the null values reflecting the fact that no garage has been built.

In [ ]:
pipeline = Pipeline([
      ( 'GarageYrBlt',  ArbitraryNumberImputer(
        arbitrary_number=0, variables='GarageYrBlt'
        ) 
      )
])
pipeline

pipeline.fit(TrainSet)
TrainSet, TestSet, df_clean = pipeline.transform(TrainSet), pipeline.transform(TestSet),pipeline.transform(df_clean)

In [ ]:
EvaluateMissingData(TrainSet)

In [ ]:
EvaluateMissingData(df_clean)

As we can see after our previous two cells we have cleaned all the data and have no null values left in our data.

___

### Comparison 

Now that we have removed all null values in our cleaned dataframe, train and test sets. we can utilise the DataCleaningEffect function that we defined earlier on in this notebook.
* This function plots our original dataframe against our clean dataframe so we can see the effect of our data cleaning.

In [ ]:
DataCleaningEffect(df_original=df,
                   df_cleaned=df_clean,
                   variables_applied_with_method=null_variables)

* Looking at the effect on the distribution of these variables we can see a bit more of a peak at LotFrontage where we imputed the mean.
* The GarageYrBlt is skewed heavily due to the addition of 0 values.
* Everything else has remained very similar.

___

### Match Datatypes across df_clean and df_inherited

In [ ]:
df_clean.info()

In [ ]:
df_inherited.info()

In [ ]:
print(df_clean.select_dtypes('object').info())
print(df_clean.select_dtypes('float').info())
print(df_clean.select_dtypes('int').info())

In [ ]:
print(df_inherited.select_dtypes('object').info())
print(df_inherited.select_dtypes('float').info())
print(df_inherited.select_dtypes('int').info())

By running the above code we can see there are some discrepencies in the data types between the cleaned dataframe and inherited dataframe. We want to rectify this that the data types are like for like across dataframes.
* All the object type variables are the same so require no change.
* There are 5 float64 variables in df_clean and 7 in df_inherited.
* There there are 13 int64 variables in df_clean and 12 in df_inherited.
* For ease we will change all floats to ints. This will then be repeated in the train and test sets.

In [ ]:
for col in df_clean.select_dtypes('float').columns:
    df_clean[col] = df_clean[col].astype('int64')

for col in df_inherited.select_dtypes('float').columns:
    df_inherited[col] = df_inherited[col].astype('int64')

print(df_clean.select_dtypes('float').info())
print(df_inherited.select_dtypes('float').info())

As we can see there are now no more float variables, we can check our dataframes as a whole to make sure everthing worked as expected and the floats have become int64

In [ ]:
print(df_clean.info())
print(df_inherited.info())

With everything working as expected we will now apply the same changes to the train and test sets.

In [ ]:
for col in TrainSet.select_dtypes('float').columns:
    TrainSet[col] = TrainSet[col].astype('int64')

for col in TestSet.select_dtypes('float').columns:
    TestSet[col] = TestSet[col].astype('int64')

print(TrainSet.info())
print(TestSet.info())

___

### Finally we can save our clean dataframe and our train and test sets

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSet.csv", index=False)

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSet.csv", index=False)

In [ ]:
df_clean.to_csv("outputs/datasets/cleaned/clean_house_price_records.csv", index=False)

In [ ]:
df_inherited.to_csv("outputs/datasets/cleaned/clean_inherited_houses.csv", index=False)

___

### Conclusion

In conclusion as we have worked through this workbook we have:
* Further analysed our data to identify variables that required cleaning and gave thought to what would be the optimum method to do so.
* we split our data into a train and test set.
* We dropped varibales that we deemed unimportant to our business goal.
* We stepped through each of our variables containing null values and cleaned them, replacing null values with values that best suited the data.
* We then evaluated our data cleaning against the original data.
* Finally we ensured that the clean data frame, test and train set and the inherited dataframe all had the same data types across all variables.